In [44]:
# dependencies
from sqlalchemy import create_engine, inspect, MetaData, select, text, Table, func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Create an engine that can talk to the database
engine = create_engine("sqlite:///Resources/la_crime.db", connect_args={'timeout': 30}, echo=False)
conn = engine.connect()

## Checking Database Data - using SQL

In [6]:
# Query data from crime table
crime_data = conn.execute("SELECT * FROM crime")

i = 0
for record in crime_data:
    print(record)
    i += 1
    if i == 10:
        break

(180512192, 2018, 6, 'June', '31-40', 'Hispanic', 'Female', 33.7932, -118.2648, 5, 626, 501)
(181211151, 2018, 4, 'April', '51-60', 'Hispanic', 'Female', 33.9891, -118.3167, 12, 624, 101)
(180220154, 2018, 9, 'September', '61-70', 'Hispanic', 'Female', 34.0529, -118.2733, 2, 624, 502)
(190909830, 2018, 10, 'October', '31-40', 'Hispanic', 'Female', 34.1867, -118.4181, 9, 440, 502)
(181113371, 2018, 7, 'July', '71+', 'White', 'Male', 34.1008, -118.2665, 11, 440, 501)
(182019832, 2018, 8, 'August', '31-40', 'Asian', 'Female', 34.0588, -118.2941, 20, 440, 502)
(181421617, 2018, 10, 'October', '31-40', 'White', 'Male', 34.029, -118.4032, 14, 480, 707)
(180810320, 2018, 5, 'May', '51-60', 'White', 'Male', 34.0711, -118.46700000000001, 8, 649, 501)
(180106451, 2018, 2, 'February', '21-30', 'Hispanic', 'Male', 34.0522, -118.2434, 1, 624, 102)
(180413435, 2018, 3, 'March', '11-20', 'Hispanic', 'Female', 34.0299, -118.2037, 4, 860, 501)


In [7]:
# Query data from crime type table
crime_type_data = conn.execute("SELECT * FROM crime_type")

i = 0
for record in crime_type_data:
    print(record)
    i += 1
    if i == 10:
        break

(626, 'Domestic Violence')
(624, 'Assault')
(440, 'Theft, Robbery, or Burglary')
(480, 'Bike Theft')
(649, 'Forgery')
(860, 'Sexual Assualt or Rape')
(421, 'Burglary from Vehicle')
(740, 'Vandalism')
(341, 'Theft, Robbery, or Burglary')
(888, 'Trespassing')


In [8]:
# Query data from crime table
area_data = conn.execute("SELECT * FROM area")

i = 0
for record in area_data:
    print(record)
    i += 1
    if i == 10:
        break

(5, 'Harbor')
(12, '77th Street')
(2, 'Rampart')
(9, 'Van Nuys')
(11, 'Northeast')
(20, 'Olympic')
(14, 'Pacific')
(8, 'West LA')
(1, 'Central')
(4, 'Hollenbeck')


In [9]:
# Query data from crime table
premise_data = conn.execute("SELECT * FROM premise")

i = 0
for record in premise_data:
    print(record)
    i += 1
    if i == 10:
        break

(501, 'Residence')
(101, 'Freeway/Street')
(502, 'Residence')
(707, 'Residence')
(102, 'Sidewalk')
(103, 'Freeway/Street')
(108, 'Parking Lot/Garage')
(404, 'Store')
(203, 'Business')
(109, 'Park')


## Joining the Tables

In [47]:
# creating database session
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine=engine, reflect=True)
Base.classes.keys()

['la_crime_joined']

In [48]:
# Collect the names of tables within the database
inspector = inspect(engine)
inspector.get_table_names()

['area', 'crime', 'crime_type', 'la_crime', 'la_crime_joined', 'premise']

In [12]:
# Using the inspector to print the column names within the 'crime' table and its types
columns = inspector.get_columns('crime')
for column in columns:
    print(column["name"], column["type"])

incident_id INTEGER
year INTEGER
month INTEGER
month_name TEXT
victim_age TEXT
victim_ethnicity TEXT
victim_gender TEXT
lat REAL
lon REAL
area_code INTEGER
crime_code INTEGER
premise_code INTEGER


In [49]:
# Reflect the database tables
metadata =  MetaData(bind=engine)
metadata.reflect()
crime_table = metadata.tables['crime']
area_table = metadata.tables['area']
crime_type_table = metadata.tables['crime_type']
premise_table = metadata.tables['premise']
crime_table

Table('crime', MetaData(bind=Engine(sqlite:///Resources/la_crime.db)), Column('incident_id', INTEGER(), table=<crime>), Column('year', INTEGER(), table=<crime>), Column('month', INTEGER(), table=<crime>), Column('month_name', TEXT(), table=<crime>), Column('victim_age', TEXT(), table=<crime>), Column('victim_ethnicity', TEXT(), table=<crime>), Column('victim_gender', TEXT(), table=<crime>), Column('lat', REAL(), table=<crime>), Column('lon', REAL(), table=<crime>), Column('area_code', INTEGER(), table=<crime>), Column('crime_code', INTEGER(), table=<crime>), Column('premise_code', INTEGER(), table=<crime>), schema=None)

In [50]:
# Select the crime table
crime_type_tb = engine.execute(select([crime_type_table]))

# Get all the rows
rows = crime_type_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

49
(626, 'Domestic Violence')


In [51]:
# Select the crime type table
crime_tb = engine.execute(select([crime_table]))

# Get all the rows
rows = crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

757085
(180512192, 2018, 6, 'June', '31-40', 'Hispanic', 'Female', 33.7932, -118.2648, 5, 626, 501)


In [52]:
# Select the area table
area_tb = engine.execute(select([area_table]))

# Get all the rows
rows = area_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

21
(5, 'Harbor')


In [53]:
# Select the premise table
premise = engine.execute(select([premise_table]))

# Get all the rows
rows = premise.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

93
(501, 'Residence')


In [32]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")

# Execute the SQL query
la_crime = engine.execute(sql)

In [33]:
# Print the column names
print(f"Columns: {la_crime.keys()}")

Columns: ['incident_id', 'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_code', 'area_name', 'premise_code', 'premise', 'crime_code', 'crime']


In [34]:
# Get all the rows
rows = la_crime.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

757085
(180512192, 2018, 6, 'June', '31-40', 'Hispanic', 'Female', 33.7932, -118.2648, 5, 626, 501, 5, 'Harbor', 501, 'Residence', 626, 'Domestic Violence')


In [21]:
# print first 10 rows
i = 0
for row in rows:
    print(row)
    if i == 10:
        break
    i += 1

(180512192, 2018, 6, 'June', '31-40', 'Hispanic', 'Female', 33.7932, -118.2648, 5, 626, 501, 5, 'Harbor', 501, 'Residence', 626, 'Domestic Violence')
(181211151, 2018, 4, 'April', '51-60', 'Hispanic', 'Female', 33.9891, -118.3167, 12, 624, 101, 12, '77th Street', 101, 'Freeway/Street', 624, 'Assault')
(180220154, 2018, 9, 'September', '61-70', 'Hispanic', 'Female', 34.0529, -118.2733, 2, 624, 502, 2, 'Rampart', 502, 'Residence', 624, 'Assault')
(190909830, 2018, 10, 'October', '31-40', 'Hispanic', 'Female', 34.1867, -118.4181, 9, 440, 502, 9, 'Van Nuys', 502, 'Residence', 440, 'Theft, Robbery, or Burglary')
(181113371, 2018, 7, 'July', '71+', 'White', 'Male', 34.1008, -118.2665, 11, 440, 501, 11, 'Northeast', 501, 'Residence', 440, 'Theft, Robbery, or Burglary')
(182019832, 2018, 8, 'August', '31-40', 'Asian', 'Female', 34.0588, -118.2941, 20, 440, 502, 20, 'Olympic', 502, 'Residence', 440, 'Theft, Robbery, or Burglary')
(181421617, 2018, 10, 'October', '31-40', 'White', 'Male', 34.029

## Creating the final table for running queries 

In [35]:
# creating database session
session = Session(engine)
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

['la_crime_joined']

In [3]:
# Define the la_crime class , which will have a one-to-one relationship with the Area, premise and crime type tables
class CrimeJoined(Base):
    __tablename__ = 'la_crime_joined'
    incident_id = Column(Integer, primary_key=True)
    year = Column(Integer)
    month = Column(Integer)
    month_name = Column(String)
    victim_age = Column(String)
    victim_ethnicity = Column(String)
    victim_gender = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    area_code = Column(Integer)
    crime_code = Column(Integer)
    premise_code = Column(Integer)
    area_name = Column(String, nullable=False)
    crime = Column(String, nullable=False)
    premise = Column(String, nullable=False)

In [4]:
# create the table in the database
metadata.create_all(bind=engine, checkfirst=True)

In [54]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")
# Execute the SQL query
la_crime = engine.execute(sql)
rows = la_crime.fetchall()
print(len(rows))

# loop over the result set and insert each row into the new table
for row in rows:
    new_row = CrimeJoined(incident_id = row.incident_id, 
                       year = row.year,
                       month = row.month,
                       month_name = row.month_name,
                       victim_age = row.victim_age, 
                       victim_ethnicity = row.victim_ethnicity,
                       victim_gender = row. victim_gender,
                       lat = row.lat,
                       lon = row.lon,
                       area_code = row.area_code,
                       crime_code = row.crime_code, 
                       premise_code= row.premise_code,
                       area_name = row.area_name, 
                       crime = row.crime,
                       premise = row.premise)
    session.add(new_row)

# commit the changes
session.commit()

757085


UnmappedInstanceError: Class '__main__.CrimeJoined' is not mapped

In [39]:
inspector.get_table_names()

['area', 'crime', 'crime_type', 'la_crime', 'la_crime_joined', 'premise']

In [40]:
# Select the la_crime table
la_crime_tb = engine.execute(select([metadata.tables['la_crime_joined']]))

In [41]:
# Get all the rows
rows = la_crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

0


IndexError: list index out of range

## Data Queries - using SQLAlchamey (copy to flask app to create functions)

use the table named la_crime_tb for all queries

In [ ]:
group_by(la_crime_tb.year).

In [24]:
# Data for overall line chart by year - number of total crimes
sel = [la_crime_tb.year, la_crime_tb.crime]
all_crimes_by_year = session.query(*sel).all()
all_crimes_by_year

AttributeError: 'ResultProxy' object has no attribute 'year'

In [ ]:
# Data for overall line chart by year - number of total crimes
sel = [LA_Crime.year, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(LA_Crime.year == 2018).\   # in the flask app replace the year 2018 with selected_year var
    filter(LA_Crime.crime == 'Domestic Violence').\   # in the flask app replace 'Domestic Violence' with selected_crime var
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages


num_rows = session.query(MyTable).count()

'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_code', 'area_name', 'premise_code', 'premise', 'crime_code', 'crime'